### Consignes et aide au chargement des données
Application empirique, enquête emploi

Formation Econométrie en Grande Dimension, Insee

14/09/2020

Jérémy L'Hour

### Consignes
Cet exercice a pour but d'aider à comprendre les concepts vus en cours (Sections 1 - 3) sur un exemple empirique, utilisant des données réelles, issues de l'Enquête Emploi.

Pour cela, on cherche à connaitre l'effet du niveau de diplôme sur le niveau de salaire, en contrôlant d'une multitude de caractéristiques. Plus précisément, on souhaite construire un intervalle de confiance ayant de bonnes propriétés autour de la valeur estimée de ces effets.

L'idée est de mettre en oeuvre les méthodes de sélection de variables (e.g. Lasso) de façon autonome, tout en se posant les bonnes questions sur la procédure à suivre, à la fois pour la mise en oeuvre des algorithmes de ML, mais aussi pour la construction de l'intervalle de confiance (méthode de la double  sélection).

L'application est décrite en détails dans la Section 3.7. Des éléments de correction sont donnés par le notebook de l'application, "ApplicationEnqueteEmploi.ipynb".

### Aide au chargement des données

In [9]:
### Liste des paquets nécessaires
rm(list=ls())
package_list = c('aws.s3','glmnet','fastDummies','caret')

In [10]:
### Si besoin d'installation, sinon passer à la vignette suivante
lapply(package_list, install.packages, character.only = TRUE)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependency ‘shape’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

In [11]:
### Chargement des paquets
lapply(package_list, require, character.only = TRUE)

Loading required package: glmnet

Loading required package: Matrix

Loaded glmnet 4.0-2



[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

In [13]:
### Chargement des données depuis la plate-forme innovation
bucket = "https://minio.stable.innovation.insee.eu/groupe-1006"
files = get_bucket(bucket = bucket, prefix = "grandedim/")
save_object(bucket = bucket,  prefix = "grandedim/", object="grandedim/data_ee.Rda" ,file="grandedim/data_ee.Rda")

ERROR: Error in curl::curl_fetch_memory(url, handle = handle): SSL: no alternative certificate subject name matches target host name 'us-east-1.minio.stable.innovation.insee.eu'


In [17]:
### Chargement des données dans R
load("data_ee.Rda")

data_use = data_ee

print(dim(data_use))

[1] 15000   718


In [18]:
# Label des variables à utiliser
outcome = "LOG_SAL"
X_1_names = "DIP"
names_continuous = c("AG", "AG_2", "ANCENTR","HHC","NBENFIND")
names_categorical = c("SEXE","APPDIP","SANTGEN","ADMHAND","CATAU2010", "CHPUB","CHRON",
                      "COMSAL","COURED","CSPM","CSPP","FORDAT","DESC","IMMI","DUHAB","ENFRED","SPE",
                      "MAISOC","MATRI","NAT14","NBAGEENFA","NBENFA1","NBENFA10","NBENFA15","NBENFA18","QP","REG","SO","SOIRC","TYPMEN21")
X_2_names = c(names_continuous,names_categorical)

In [21]:
# "Y" (outcome) -- log du salaire mensuel
Y = data_use[,outcome]
summary(Y)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  3.401   7.131   7.415   7.389   7.730  11.290 

In [28]:
# "X_1" (variables d'intérêt -- celles dont on veut connaitre l'effet) -- niveau d'éducation
dip_ref = data.frame("ID" = c("10","12","22","21","30","31","32","33","41","42","43","44","50","60","70"),
                     "Diplome" = c("Master (recherche ou professionnel), DEA, DESS, Doctorat",
              "Ecoles niveau licence et au-delà",
              "Maîtrise (M1)",
              "Licence (L3)",
              "DEUG",
              "DUT, BTS",
              "Autre diplôme (niveau bac+2)",
              "Paramédical et social (niveau bac+2)",
              "Baccalauréat général",
              "Bac technologique",
              "Bac professionnel",
              "Brevet de technicien, brevet professionnel",
              "CAP, BEP",
              "Brevet des collèges",
              "Certificat d'études primaires")
                     )

dip_freq = data.frame("Frequence" = round(100 * table(as.factor(data_use[,X_1_names])) / nrow(data_use), digits=2))

print(merge(dip_ref, dip_freq, by.x="ID", by.y="Frequence.Var1", sort = TRUE))

X_1 = model.matrix(~. - 1, data = data.frame("EDUC"=as.factor(data_use[,X_1_names])), contrasts.arg = "EDUC")
X_1 = X_1[,1:(ncol(X_1)-1)] # On enlève la modalité "sans diplôme" pour éviter les problèmes de colinéarité.

   ID                                                  Diplome Frequence.Freq
1  10 Master (recherche ou professionnel), DEA, DESS, Doctorat           7.82
2  12                         Ecoles niveau licence et au-delà           5.13
3  21                                             Licence (L3)           6.16
4  22                                            Maîtrise (M1)           3.27
5  30                                                     DEUG           1.13
6  31                                                 DUT, BTS          10.66
7  32                             Autre diplôme (niveau bac+2)           0.63
8  33                     Paramédical et social (niveau bac+2)           2.81
9  41                                     Baccalauréat général           4.97
10 42                                        Bac technologique           5.55
11 43                                        Bac professionnel           6.53
12 44               Brevet de technicien, brevet professionnel  

Warning message in model.matrix.default(~. - 1, data = data.frame(EDUC = as.factor(data_use[, :
“non-list contrasts argument ignored”


In [29]:
# "X_2" (variables de contrôle -- celles qu'il faut sélectionner)
one_hot_category = dummy_cols(data_use[,names_categorical], remove_most_frequent_dummy=TRUE, remove_selected_columns=TRUE) # on transforme les variables catégorielles en variables binaires
X_2 = as.matrix(cbind(data_use[, names_continuous], one_hot_category))
X_2 = X_2[,!duplicated(t(X_2))] # On enlève les colonnes dupliquées
colinear = caret::findLinearCombos(cbind(X_1,X_2,rep(1,nrow(X_2)))) 
suppr = colinear$remove-ncol(X_1) # recalage par rapport à l'indice de X_2
X_2 = X_2[,-suppr] # On enlève les colonnes qui créent de la multicolinéairité, avec l'inclusion de X_1 et une constante

In [30]:
# Identifiants clustering -- pas forcément nécessaire ici
ID_menage = data_use[,"IDENT"] # Identifiant du ménage, pour cluster dans les écart-types.

coef_names = paste("X_1",colnames(X_1),sep="") # nom des variables d'intérêt
n = nrow(X_2); p = ncol(X_2)

remove(data_use, one_hot_category)

In [33]:
### Exemple de régression du log du salaire sur le niveau de diplôme
reg_simple = lm(Y ~ X_1)
summary(reg_simple)
tau_simple = reg_simple$coefficients[coef_names] # on collecte la valeur des coefficients dans ce vecteur


Call:
lm(formula = Y ~ X_1)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.1346 -0.2235  0.0532  0.3135  3.4698 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  7.02964    0.01301 540.240  < 2e-16 ***
X_1EDUC10    0.79031    0.02053  38.496  < 2e-16 ***
X_1EDUC12    0.90035    0.02354  38.254  < 2e-16 ***
X_1EDUC21    0.51620    0.02212  23.333  < 2e-16 ***
X_1EDUC22    0.71588    0.02780  25.749  < 2e-16 ***
X_1EDUC30    0.45279    0.04370  10.362  < 2e-16 ***
X_1EDUC31    0.49767    0.01882  26.440  < 2e-16 ***
X_1EDUC32    0.42282    0.05730   7.380 1.67e-13 ***
X_1EDUC33    0.56889    0.02953  19.266  < 2e-16 ***
X_1EDUC41    0.22977    0.02380   9.655  < 2e-16 ***
X_1EDUC42    0.35592    0.02290  15.542  < 2e-16 ***
X_1EDUC43    0.23758    0.02171  10.942  < 2e-16 ***
X_1EDUC44    0.29495    0.03109   9.487  < 2e-16 ***
X_1EDUC50    0.25171    0.01578  15.955  < 2e-16 ***
X_1EDUC60    0.12183    0.02372   5.136 2.84e-07 ***
X_1EDUC70   -0